In [1]:
# %%capture
# pip install mne

!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# import mne
import pywt
import wandb

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DWT_file = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/DWT/DWT.npz'
DWT = np.load(DWT_file, allow_pickle=True)  # Allow loading pickled object arrays
X = DWT['X'].astype('float64')
Y = DWT['Y'].astype('float64')
group = DWT['G'].astype('float64')
X= np.moveaxis(X,1,2)
X.shape

(5834, 52, 29)

#Split the data

In [5]:
X_reshaped = X.reshape(X.shape[0],-1)
X_reshaped.shape



#Scale data
scaler = StandardScaler()
scaled_data_reshaped = scaler.fit_transform(X_reshaped)

# Reshape back to original shape if necessary
scaled_data = scaled_data_reshaped.reshape(X.shape)


In [6]:
# Configuration
n_splits = 5  # Number of folds
random_state = 42
shuffle = True

# Initialize KFold
kf = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

# Create a placeholder array for fold assignments
fold_assignments = np.empty(len(Y), dtype=int)

# Split the data using kf.split. Note: kf.split does not need Y for splitting, but it's kept here for consistency
for fold, (train_idx, test_idx) in enumerate(kf.split(scaled_data, Y)):
    # Assign fold number
    fold_assignments[test_idx] = fold

# Example: To get the train and test sets for the first fold
fold_number = 0  # Choose which fold to use (0 to n_splits-1)
train_idx = np.where(fold_assignments != fold_number)[0]
test_idx = np.where(fold_assignments == fold_number)[0]

X_train, X_test = scaled_data[train_idx], scaled_data[test_idx]
Y_train, Y_test = Y[train_idx], Y[test_idx]

In [7]:
X_train.shape,Y_train.shape,X_test.shape

((4667, 52, 29), (4667,), (1167, 52, 29))

#CNN Model

In [8]:

!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


This is with using only the data with out any transformations applied. So This is just plain Deep learning.

In [9]:
X_train.shape, Y_train.shape,X_test.shape

((4667, 52, 29), (4667,), (1167, 52, 29))

In [10]:
from wandb.keras import WandbCallback

In [11]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import callbacks
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import  Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,Flatten,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D,ReLU,Conv2D,MaxPool2D,GlobalAveragePooling2D,AveragePooling2D,Bidirectional, LSTM, concatenate
from tensorflow.keras.regularizers import l2
from keras.layers import ConvLSTM1D, LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM

# early_stopping = callbacks.EarlyStopping(
#     min_delta=0.001,
#     patience=10,
#     restore_best_weights=True
# )

from keras import backend as K

def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)

        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))

        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from keras_radam import RAdam

checkpoint = ModelCheckpoint("/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Mobilenet1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = .25,
                              patience = 2,
                              verbose = 1)


# tf_callback = tf.keras.callbacks.TensorBoard(
#                             log_dir="/content/drive/MyDrive/DeepLearning/TomatoLeaf/logs",
#                             histogram_freq=1,
#                             write_graph=True,
#                             write_images=True,
#                             update_freq="epoch"
#                             )

# we put our call backs into a callback list
wandb.init(project="Epileptic Seizure", entity="RaihanRabby", dir = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb')
callbacks = [checkpoint,reduce_lr,WandbCallback()]

accuracy,▁▆▆▇████████████████▁▆▇█████████████████
epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇█▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇██
loss,█▄▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,███▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▃▇▇█████████████████▁▇▅█████████████████
val_loss,▄▂█▃▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▁▁▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
accuracy,0.97364
best_epoch,3
best_val_loss,0.02165
epoch,49
loss,0.00772


In [17]:
 # Replace 'your_loss_module' with the actual module name where focal_loss is defined
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout, GRU
from keras.optimizers import Adam
from keras.backend import clear_session

def build_sequential_model(input_shape):
    clear_session()  # Clear the previous Keras session to avoid clutter
    model = Sequential()

    # Conv1D layers
    model.add(Conv1D(filters=64, kernel_size=5, strides=1, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    # model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    # model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(MaxPool1D(pool_size=4, padding='same'))
    model.add(MaxPool1D(pool_size=4, padding='same'))
    model.add(MaxPool1D(pool_size=4, padding='same'))

    # GRU layers
    model.add(GRU(64, return_sequences=True))
    model.add(GRU(32, return_sequences=False))  # The last GRU layer should not return sequences to flatten the output

    # Dropout for regularization
    model.add(Dropout(0.25))

    # Dense layers for classification or regression
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Adjust based on your application

    # Compile the model
    opt = Adam(learning_rate=0.0001, beta_1=0.09, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer=opt, loss=focal_loss(), metrics=['accuracy'])

    return model

# Usage
Model = build_sequential_model(input_shape=(52, 29))
Model.fit(X_train, Y_train, epochs=50, batch_size=64, callbacks=[callbacks], validation_data=(X_test, Y_test), shuffle=True)

Epoch 1/50
73/73 [==============================] - ETA: 0s - loss: 0.0546 - accuracy: 0.6925
Epoch 1: val_loss improved from inf to 0.03980, saving model to /content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Mobilenet1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb/wandb/run-20240228_185902-cotkt2hj/files/model-best)... Done. 0.1s


73/73 [==============================] - 27s 281ms/step - loss: 0.0546 - accuracy: 0.6925 - val_loss: 0.0398 - val_accuracy: 0.8355 - lr: 1.0000e-04
Epoch 2/50
73/73 [==============================] - ETA: 0s - loss: 0.0351 - accuracy: 0.8410
Epoch 2: val_loss improved from 0.03980 to 0.03079, saving model to /content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Mobilenet1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb/wandb/run-20240228_185902-cotkt2hj/files/model-best)... Done. 0.1s


73/73 [==============================] - 23s 320ms/step - loss: 0.0351 - accuracy: 0.8410 - val_loss: 0.0308 - val_accuracy: 0.8980 - lr: 1.0000e-04
Epoch 3/50
73/73 [==============================] - ETA: 0s - loss: 0.0279 - accuracy: 0.8864
Epoch 3: val_loss improved from 0.03079 to 0.02429, saving model to /content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Mobilenet1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb/wandb/run-20240228_185902-cotkt2hj/files/model-best)... Done. 0.1s


73/73 [==============================] - 21s 288ms/step - loss: 0.0279 - accuracy: 0.8864 - val_loss: 0.0243 - val_accuracy: 0.9075 - lr: 1.0000e-04
Epoch 4/50
73/73 [==============================] - ETA: 0s - loss: 0.0252 - accuracy: 0.8995
Epoch 4: val_loss did not improve from 0.02429
73/73 [==============================] - 11s 143ms/step - loss: 0.0252 - accuracy: 0.8995 - val_loss: 0.0276 - val_accuracy: 0.9169 - lr: 1.0000e-04
Epoch 5/50
73/73 [==============================] - ETA: 0s - loss: 0.0235 - accuracy: 0.9087
Epoch 5: val_loss improved from 0.02429 to 0.02340, saving model to /content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Mobilenet1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb/wandb/run-20240228_185902-cotkt2hj/files/model-best)... Done. 0.1s


73/73 [==============================] - 23s 320ms/step - loss: 0.0235 - accuracy: 0.9087 - val_loss: 0.0234 - val_accuracy: 0.9066 - lr: 1.0000e-04
Epoch 6/50
73/73 [==============================] - ETA: 0s - loss: 0.0223 - accuracy: 0.9177
Epoch 6: val_loss did not improve from 0.02340
73/73 [==============================] - 11s 151ms/step - loss: 0.0223 - accuracy: 0.9177 - val_loss: 0.0246 - val_accuracy: 0.9117 - lr: 1.0000e-04
Epoch 7/50
73/73 [==============================] - ETA: 0s - loss: 0.0205 - accuracy: 0.9244
Epoch 7: val_loss did not improve from 0.02340

Epoch 7: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
73/73 [==============================] - 10s 130ms/step - loss: 0.0205 - accuracy: 0.9244 - val_loss: 0.0237 - val_accuracy: 0.9083 - lr: 1.0000e-04
Epoch 8/50
73/73 [==============================] - ETA: 0s - loss: 0.0167 - accuracy: 0.9400
Epoch 8: val_loss did not improve from 0.02340
73/73 [==============================] - 11s 149ms/st

In [ ]:
pred = Model.predict(X_test)

37/37 [==============================] - 1s 24ms/step
